In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch

In [211]:
!git clone https://github.com/pantherofficial404/Recommander-System-RBM-.git

fatal: destination path 'Recommander-System-RBM-' already exists and is not an empty directory.


In [0]:
movieDataset = pd.read_csv("Recommander-System-RBM-/ml-1m/movies.dat",sep="::",encoding="latin-1",engine="python",header=None)

In [213]:
movieDataset.columns = ["No",'MovieName','Category']
movieDataset.head()

,No,MovieName,Category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
ratingDataset = pd.read_csv("Recommander-System-RBM-/ml-1m/ratings.dat",sep="::",encoding="latin-1",engine="python",header=None)

In [215]:
ratingDataset.columns = ["UserId","MovieId","Rating","Timestamp"]
ratingDataset.head()

,UserId,MovieId,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [0]:
userDataset = pd.read_csv("Recommander-System-RBM-/ml-1m/users.dat",sep="::",encoding="latin-1",engine="python",header=None)

In [217]:
userDataset.columns = ["UserId","Gender","Age","Quotes","JobCodes"]
userDataset.head()

,UserId,Gender,Age,Quotes,JobCodes
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [218]:
trainingDataset = pd.read_csv("Recommander-System-RBM-/ml-100k/u1.base",delimiter="\t")
trainingDataset = np.array(trainingDataset,"int")
trainingDataset[0]

array([        1,         2,         3, 876893171])

In [219]:
testingDataset = pd.read_csv("Recommander-System-RBM-/ml-100k/u1.test",delimiter="\t")
testingDataset = np.array(testingDataset,"int")
testingDataset.shape

(19999, 4)

In [220]:
numberOfUsers = int(max(max(testingDataset[:,0]),max(trainingDataset[:,0])))
print(numberOfUsers)

943


In [221]:
numberOfMovies = int(max(max(testingDataset[:,1]),max(trainingDataset[:,1])))
print(numberOfMovies)

1682


In [0]:
def converstion(dataset):
  newDataset = []
  for eachUser in range(1,numberOfUsers+1):
    movieId = dataset[:,1][dataset[:,0]==eachUser]
    movieRating = dataset[:,2][dataset[:,0]==eachUser]
    ratings = np.zeros(numberOfMovies)
    ratings[movieId-1] = movieRating
    newDataset.append(list(ratings))
  return newDataset

In [0]:
trainingDataset = converstion(trainingDataset)

In [0]:
testingDataset = converstion(testingDataset)

In [0]:
trainingDataset = torch.FloatTensor(trainingDataset)
testingDataset = torch.FloatTensor(testingDataset)

In [0]:
trainingDataset[trainingDataset==0] = -1
trainingDataset[trainingDataset==1] = 0
trainingDataset[trainingDataset==2] = 0
trainingDataset[trainingDataset >=3] = 1

In [0]:
testingDataset[testingDataset==0] = -1
testingDataset[testingDataset==1] = 0
testingDataset[testingDataset==2] = 0
testingDataset[testingDataset>=3] = 1

# Architecture Of Restricted Boltzmann Machine

In [0]:
class RBM():
  
  def __init__(self,visibleNodes,hiddenNodes):
    self.W = torch.randn(hiddenNodes,visibleNodes)
    self.a = torch.randn(1,hiddenNodes)
    self.b = torch.randn(1,visibleNodes)
    
  def sampleH(self,x):
    wx = torch.mm(x,self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v,torch.bernoulli(p_h_given_v)
  
  def sampleV(self,y):
    wy = torch.mm(y,self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h,torch.bernoulli(p_v_given_h)
  
  def train(self,v0,vk,ph0,phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0-vk),0)
    self.a += torch.sum((ph0-phk),0)

# Creating RBM Object

In [229]:
nv = len(trainingDataset[0])
print("Visible Nodes:-",nv)
nh = 100
print("Hidden Nodes :-",nh)
batchSize = 100
print("Batch Size :-",batchSize)
rbm = RBM(nv,nh)

Visible Nodes:- 1682
Hidden Nodes :- 100
Batch Size :- 100


In [230]:
print("Weights Initalized :- ",rbm.W)
print("Bias For Hidden Layer :- ",rbm.a)
print("Bias For Visible Layer :-",rbm.b)

Weights Initalized :-  tensor([[ 0.9301, -1.1242, -0.7816,  ...,  0.4769, -1.0087,  0.9059],
        [ 0.0560,  0.1394,  1.1202,  ...,  0.9906,  0.7042,  0.3615],
        [ 1.2417, -0.4303, -0.1722,  ..., -0.8343, -0.7423, -2.1192],
        ...,
        [-0.1252, -1.4678, -2.6928,  ...,  1.4371, -1.8886,  1.4382],
        [ 1.8453,  0.9471, -1.0218,  ..., -0.5217,  0.5059, -1.3369],
        [ 0.4019,  0.3515,  0.6460,  ..., -0.2708,  0.5093,  0.2836]])
Bias For Hidden Layer :-  tensor([[ 0.8789, -0.6792, -0.1912, -0.2998,  1.3332, -0.4275, -0.4483,  0.5865,
         -1.3010,  0.6818,  0.3059, -0.0734,  1.0572,  0.4060, -0.9297, -0.1372,
          1.0562, -0.2082,  0.5514,  0.0048, -0.0809, -0.4425, -0.4409,  0.4529,
         -0.7482,  0.3859,  0.5992, -0.6821,  0.6502, -1.0698,  0.0736, -0.3623,
          1.3259, -1.1378,  0.7511, -0.5309, -0.3630,  1.2903,  1.9792,  0.1572,
          0.4208, -2.2103, -0.0289, -0.1759, -0.0205,  0.1433, -0.4506, -0.3912,
         -0.1159, -0.7839, -1.1

In [231]:
epochs = 10
for epoch in range(1,epochs+1):
  train_loss = 0
  s = 0.0
  for users in range(0,numberOfUsers-batchSize,batchSize):
    vk = trainingDataset[users:users+batchSize]
    v0 = trainingDataset[users:users+batchSize]
    ph0,_ = rbm.sampleH(v0)
    for k in range(10):
      _,hk = rbm.sampleH(vk)
      _,vk = rbm.sampleV(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sampleH(vk)
    rbm.train(v0,vk,ph0,phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    s += 1
  print("Epohcs :-",epoch,"Loss :-",str(train_loss/s))

Epohcs :- 1 Loss :- tensor(0.3633)
Epohcs :- 2 Loss :- tensor(0.2540)
Epohcs :- 3 Loss :- tensor(0.2489)
Epohcs :- 4 Loss :- tensor(0.2514)
Epohcs :- 5 Loss :- tensor(0.2486)
Epohcs :- 6 Loss :- tensor(0.2495)
Epohcs :- 7 Loss :- tensor(0.2507)
Epohcs :- 8 Loss :- tensor(0.2466)
Epohcs :- 9 Loss :- tensor(0.2457)
Epohcs :- 10 Loss :- tensor(0.2477)


In [233]:
test_loss = 0
s = 0.0
for users in range(0,numberOfUsers):
  v = trainingDataset[users:users+1]
  vt = testingDataset[users:users+1]
  if len(vt[vt>=0])>0:
    _,h = rbm.sampleH(v)
    _,v = rbm.sampleV(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    s += 1
print("Test Loss :-",str(test_loss/s))

Test Loss :- tensor(0.2422)
